In [1]:
!which pip # expect pip to be inside .../Versions/3.x/bin/pip
!echo
!which python
!echo
!which python3 # make sure python3 is a thing
!echo
!spacy # make sure spacy is a thing

/Library/Frameworks/Python.framework/Versions/3.7/bin/pip

/usr/bin/python

/Library/Frameworks/Python.framework/Versions/3.7/bin/python3

/bin/sh: /Library/Frameworks/Python.framework/Versions/3.7/bin/spacy: Permission denied


In [2]:
!pip install spacy

In [3]:
!python3 -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
import numpy as np
import pandas as pd
import os
import spacy

In [5]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

In [6]:
# # Load English tokenizer, tagger, parser, NER and word vectors
# nlp = spacy.load("en_core_web_sm")

# # Process whole documents
# text = ("When Sebastian Thrun started working on self-driving cars at "
#         "Google in 2007, few people outside of the company took him "
#         "seriously. “I can tell you very senior CEOs of major American "
#         "car companies would shake my hand and turn away because I wasn’t "
#         "worth talking to,” said Thrun, in an interview with Recode earlier "
#         "this week.")
# doc = nlp(text)

# tok = doc[0]

# # https://spacy.io/api/token

# print(tok)

# print("doc",[(token.text, token.is_stop) for token in doc if not token.is_stop or not token.is_punct])

# # Analyze syntax
# print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
# print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# # Find named entities, phrases and concepts
# for entity in doc.ents:
#     print(entity.text, entity.label_)

In [7]:
DATA_DIR = "state-of-the-state"
FILENAME = "index.csv"
SPEECH_DIR_NAME = "speeches"
TABLE_PATH = os.path.join(DATA_DIR, FILENAME)
SPEECH_DIR = os.path.join(DATA_DIR, SPEECH_DIR_NAME)
print("DATA_DIR:\t", os.listdir(DATA_DIR))
print("SPEECH_DIR[:2]:\t", os.listdir(SPEECH_DIR)[:2])
print("TABLE_PATH:\t", TABLE_PATH)
print("SPEECH_DIR:\t", SPEECH_DIR)

DATA_DIR:	 ['words.csv', 'speeches', 'README.md', 'index.csv']
SPEECH_DIR[:2]:	 ['Minnesota_SOTS.txt', 'Indiana_SOTS.txt']
TABLE_PATH:	 state-of-the-state/index.csv
SPEECH_DIR:	 state-of-the-state/speeches


In [8]:
# read the csv for info on the data
df = pd.read_csv(TABLE_PATH)
df.head()

,state,governor,party,filename,url
0,Alabama,Kay Ivey,R,Alabama_SOTS.txt,https://governor.alabama.gov/remarks-speeches/...
1,Alaska,Mike Dunleavy,R,Alaska_SOTS.txt,https://gov.alaska.gov/newsroom/2019/01/22/201...
2,Arizona,Doug Ducey,R,Arizona_SOTS.txt,https://azgovernor.gov/governor/news/2019/01/g...
3,Arkansas,Asa Hutchinson,R,Arkansas_SOTS.txt,https://governor.arkansas.gov/news-media/speec...
4,California,Gavin Newsom,D,California_SOTS.txt,https://www.gov.ca.gov/2019/02/12/state-of-the...


In [9]:
# dictionary mapping filenames to text content
speech_map = {}

# read the files
for fname in df['filename'].get_values():
    with open( os.path.join(SPEECH_DIR, fname) ) as f:
        speech_map[fname] = f.read()
        f.close()

In [10]:
# https://spacy.io/api/span#set_extension
# extend Span object with is_stop
from spacy.tokens import Span

# if the length is 1 and the word is a stopword
stopword_checker_for_span = lambda span: (len(span) == 1 and type(span[0]).__name__ == "Token" and span[0].is_stop)
Span.set_extension("is_stop", getter=stopword_checker_for_span)

# test ._.is_stop
assert(nlp("I")[:]._.is_stop == True)
assert(nlp("Hello world")[:]._.is_stop == False)

In [11]:
# https://spacy.io/usage/linguistic-features#noun-chunks

speech_entity_map = {}

for state in speech_map:
    doc = nlp(speech_map[state])

    entities = []
    for chunk in doc.noun_chunks:
        if (not chunk._.is_stop): # e.g. if chunk != ["it"]
            entities.append(chunk.text)
#         # uncomment else block to debug and expect the printouts to look like stopwords
#         else: 
#             print(type(chunk), chunk[0], type(chunk[0]), chunk.text)
    print(f"got noun chunks for {state}")
    speech_entity_map[state] = entities

got noun chunks for Alabama_SOTS.txt
got noun chunks for Alaska_SOTS.txt
got noun chunks for Arizona_SOTS.txt
got noun chunks for Arkansas_SOTS.txt
got noun chunks for California_SOTS.txt
got noun chunks for Colorado_SOTS.txt
got noun chunks for Connecticut_SOTS.txt
got noun chunks for Delaware_SOTS.txt
got noun chunks for Florida_SOTS.txt
got noun chunks for Georgia_SOTS.txt
got noun chunks for Hawaii_SOTS.txt
got noun chunks for Idaho_SOTS.txt
got noun chunks for Illinois_Both.txt
got noun chunks for Indiana_SOTS.txt
got noun chunks for Iowa_SOTS.txt
got noun chunks for Kansas_SOTS.txt
got noun chunks for Kentucky_SOTS.txt
got noun chunks for Louisiana_SOTS.txt
got noun chunks for Maine_SOTS.txt
got noun chunks for Maryland_SOTS.txt
got noun chunks for Massachusetts_Both.txt
got noun chunks for Michigan_SOTS.txt
got noun chunks for Minnesota_SOTS.txt
got noun chunks for Mississippi_SOTS.txt
got noun chunks for Missouri_SOTS.txt
got noun chunks for Montana_SOTS.txt
got noun chunks for

In [12]:
print(speech_entity_map['Alabama_SOTS.txt'])

['Lieutenant Governor Ainsworth', 'Pro Tempore Marsh', 'Speaker McCutcheon', 'Speaker Pro Tempore Gaston', 'members', 'the Alabama Legislature', 'Chief Justice Parker', 'justices', 'the Alabama Supreme Court', 'my fellow Alabamians', 'Mother Nature', '’s', 'the form', 'our state', 'significant devastation', 'At least 23 innocent lives', 'Young children', 'life', 'Mothers', 'Fathers', 'Friends', 'neighbors', 'times', 'the good Lord', 'His continued comfort', 'healing hands', 'special thanks', 'the emergency responders', 'local law enforcement', 'a moment', 'silence', 'many others', 'uncertainty', 'what tomorrow', 'absolute certainty', 'the resiliency', 'the people', 'Alabama', 'a time', 'our entire nation', 'these good people', 'Lee County', 'its feet', 'our 200 years', 'statehood', 'the men', 'women', 'Alabama', 'our nation', 'our country', 'defending', 'the legendary Tuskegee Airmen', 'our country', 'injustice', 'an Alabama woman', 'her seat', 'a bus', 'our country', 'man', 'the moon'

In [13]:
tokens.remove(',')

NameError: name 'tokens' is not defined

In [ ]:
tokens

In [ ]:
nltk.tokenize.word_tokenize("cannot can't apple-pie")

In [ ]:
print(type(nlp("hello world")[:]).__module__)
print(type(nlp("hello world")[:]).__name__)
print(type(nlp("hello world")[0]).__name__)
print(type(nlp("hello world")).__name__)